In [1]:
!pip install pathway bokeh panel --quiet
#installing bokeh to plot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#importing necessary libraries
import numpy as np
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn
import datetime

pn.extension()


In [10]:
#reading csv,clubbing date and time to datetime format, encoding traffic conditions to integer format and saving required date in another csv
df = pd.read_csv("/content/dataset (1).csv")


df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                 format="%d-%m-%Y %H:%M:%S")

df["TrafficConditionNearby"] = df["TrafficConditionNearby"].astype(str).str.strip().str.lower()
traffic_map = {"low": 0, "average": 1, "high": 2}
df["TrafficConditionNearby"] = df["TrafficConditionNearby"].map(traffic_map)

df["TrafficConditionNearby"] = df["TrafficConditionNearby"].astype(int)

df[["Timestamp", "SystemCodeNumber", "Occupancy", "Capacity", "QueueLength",
    "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]].to_csv("parking_stream.csv", index=False)


In [4]:
#setting up Pathway Schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: int
    IsSpecialDay: int
    VehicleType: str

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=500)


In [5]:
#Model 2 Pricing logic
BASE_PRICE = 10
VEHICLE_TYPE_WEIGHTS = {"car": 1.0, "bike": 0.6, "truck": 1.4}

@pw.udf
def calculate_raw_price(occupancy, capacity, queue_length, traffic, is_special, vehicle_type):
    occ_ratio = occupancy / capacity if capacity > 0 else 0
    queue_term = np.log1p(queue_length)
    vehicle_weight = VEHICLE_TYPE_WEIGHTS.get(vehicle_type.lower(), 1.0)

    weighted_sum = (
        6.0 * occ_ratio +
        2.0 * queue_term +
        1.5 * traffic +
        2.0 * is_special +
        1.5 * vehicle_weight
    ) - 7.5

    demand_score = 1 / (1 + np.exp(-weighted_sum)) #sigmoid transformation
    price_factor = 1.0 + 0.2 * (demand_score-0.5)

    price = BASE_PRICE * price_factor
    return round(min(20, max(5, price)), 2)


In [6]:
# Transforming each incoming row
price_table = data.with_columns(
    t = pw.this.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"),
    price = pw.declare_type(
        float,
        calculate_raw_price(
            pw.this.Occupancy,
            pw.this.Capacity,
            pw.this.QueueLength,
            pw.this.TrafficConditionNearby,
            pw.this.IsSpecialDay,
            pw.this.VehicleType
        )
    )
)


In [7]:
# Defining a custom Bokeh plotting function to visualize price over time:
def price_plotter(source, title):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title=title,
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig


In [8]:
# Generating interactive price plots for each parking lot:
plots = []

for lot in df["SystemCodeNumber"].unique():
    filtered_table = price_table.filter(pw.this.SystemCodeNumber == lot)

    plot = filtered_table.select(pw.this.t, pw.this.price).plot(
        lambda source, lot_id=lot: price_plotter(source, f"Lot {lot_id} - Real-Time Price"),
        sorting_col="t"
    )
    plots.append(plot)


layout = pn.Column(*plots)
pn.panel(layout).servable()



Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [9]:
pw.run()


Output()